# ECH Workers RS - Colab Build

This notebook builds the ECH Workers RS project on Google Colab.

**Repository**: https://github.com/justinwoo280/ech-workers-rs.git

**Branch**: master

## 1. Install Build Tools

In [ ]:
#@title Install Zig 0.13.0
%%bash
set -e

ZIG_VERSION="0.13.0"
ZIG_DIR="/opt/zig"

if [ ! -f "$ZIG_DIR/zig" ]; then
    echo "📦 Downloading Zig $ZIG_VERSION..."
    wget -q "https://ziglang.org/download/$ZIG_VERSION/zig-linux-x86_64-$ZIG_VERSION.tar.xz" -O /tmp/zig.tar.xz
    
    echo "📂 Extracting..."
    sudo mkdir -p $ZIG_DIR
    sudo tar -xf /tmp/zig.tar.xz -C $ZIG_DIR --strip-components=1
    rm /tmp/zig.tar.xz
fi

# Add to PATH
echo "export PATH=$ZIG_DIR:\$PATH" >> ~/.bashrc
export PATH=$ZIG_DIR:$PATH

$ZIG_DIR/zig version
echo "✅ Zig installed"

In [ ]:
#@title Install Rust
%%bash
set -e

if ! command -v rustc &> /dev/null; then
    echo "📦 Installing Rust..."
    curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --default-toolchain stable
fi

source $HOME/.cargo/env
rustc --version
cargo --version
echo "✅ Rust installed"

In [ ]:
#@title Install Build Dependencies
%%bash
set -e

echo "📦 Installing build dependencies..."
sudo apt-get update -qq
sudo apt-get install -y -qq cmake ninja-build clang libclang-dev pkg-config libssl-dev

echo "✅ Dependencies installed"

## 2. Clone Repository

In [ ]:
#@title Clone Repository
%%bash
set -e

cd /content

REPO_URL="https://github.com/justinwoo280/ech-workers-rs.git"
BRANCH="master"

if [ -d "ech-workers-rs" ]; then
    echo "📂 Repository already exists, pulling latest..."
    cd ech-workers-rs
    git fetch origin
    git checkout $BRANCH
    git pull origin $BRANCH
    git submodule update --init --recursive
else
    echo "📥 Cloning repository..."
    git clone --recursive -b $BRANCH $REPO_URL
    cd ech-workers-rs
fi

echo "✅ Repository cloned"
echo "Branch: $(git branch --show-current)"
ls -la

## 3. Build BoringSSL

In [ ]:
#@title Build BoringSSL
%%bash
set -e

cd /content/ech-workers-rs/zig-tls-tunnel/vendor/boringssl

if [ -f "build/libssl.a" ] && [ -f "build/libcrypto.a" ]; then
    echo "✅ BoringSSL already built"
    ls -lh build/lib*.a
else
    echo "🔨 Building BoringSSL..."
    mkdir -p build
    cd build
    
    cmake -GNinja \
        -DCMAKE_BUILD_TYPE=Release \
        -DCMAKE_C_COMPILER=clang \
        -DCMAKE_CXX_COMPILER=clang++ \
        ..
    
    ninja -j$(nproc)
    
    echo "✅ BoringSSL built"
    ls -lh lib*.a
fi

## 4. Build Zig TLS Tunnel

In [ ]:
#@title Build Zig TLS Tunnel
%%bash
set -e

export PATH=/opt/zig:$PATH

cd /content/ech-workers-rs/zig-tls-tunnel

echo "🔨 Building Zig TLS Tunnel..."
zig build -Doptimize=ReleaseFast

echo "✅ Zig TLS Tunnel built"
ls -lh zig-out/lib/

## 5. Build Rust Client

In [ ]:
#@title Build Rust Client
%%bash
set -e

source $HOME/.cargo/env
export PATH=/opt/zig:$PATH

cd /content/ech-workers-rs/ech-workers-rs

echo "🔨 Building Rust client..."
cargo build --release 2>&1 | tail -20

echo ""
echo "✅ Rust client built"
ls -lh target/release/ech-workers-rs 2>/dev/null || echo "Binary not found (check build output)"

In [ ]:
#@title Strip Binary (Reduces Size)
%%bash
set -e

cd /content/ech-workers-rs/ech-workers-rs

BINARY="target/release/ech-workers-rs"

if [ -f "$BINARY" ]; then
    echo "Before strip:"
    ls -lh $BINARY
    
    strip $BINARY
    
    echo ""
    echo "After strip:"
    ls -lh $BINARY
else
    echo "Binary not found"
fi

## 6. Download Artifacts

In [ ]:
#@title Download Built Binary
from google.colab import files
import os

binary_path = '/content/ech-workers-rs/ech-workers-rs/target/release/ech-workers-rs'

if os.path.exists(binary_path):
    size_mb = os.path.getsize(binary_path) / 1024 / 1024
    print(f"📦 Downloading binary ({size_mb:.1f} MB)...")
    files.download(binary_path)
else:
    print("❌ Binary not found. Build may have failed.")

In [ ]:
#@title Download Zig Library
from google.colab import files
import os

lib_path = '/content/ech-workers-rs/zig-tls-tunnel/zig-out/lib/libzig-tls-tunnel.a'

if os.path.exists(lib_path):
    size_kb = os.path.getsize(lib_path) / 1024
    print(f"📦 Downloading Zig library ({size_kb:.1f} KB)...")
    files.download(lib_path)
else:
    print("❌ Library not found.")

---
## 🚀 Quick Build (All-in-One)

Run all build steps in one cell:

In [ ]:
#@title 🚀 Quick Build (All Steps)
%%bash
set -e

echo "========================================"
echo "ECH Workers RS - Full Build"
echo "========================================"

REPO_URL="https://github.com/justinwoo280/ech-workers-rs.git"
BRANCH="master"

# Install Zig
ZIG_DIR="/opt/zig"
if [ ! -f "$ZIG_DIR/zig" ]; then
    echo "📦 Installing Zig..."
    wget -q "https://ziglang.org/download/0.13.0/zig-linux-x86_64-0.13.0.tar.xz" -O /tmp/zig.tar.xz
    sudo mkdir -p $ZIG_DIR
    sudo tar -xf /tmp/zig.tar.xz -C $ZIG_DIR --strip-components=1
    rm /tmp/zig.tar.xz
fi
export PATH=$ZIG_DIR:$PATH

# Install Rust
if ! command -v rustc &> /dev/null; then
    echo "📦 Installing Rust..."
    curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --default-toolchain stable
fi
source $HOME/.cargo/env

# Install dependencies
echo "📦 Installing dependencies..."
sudo apt-get update -qq
sudo apt-get install -y -qq cmake ninja-build clang libclang-dev pkg-config libssl-dev

cd /content

# Clone if needed
if [ ! -d "ech-workers-rs" ]; then
    echo "📥 Cloning repository..."
    git clone --recursive -b $BRANCH $REPO_URL
fi
cd ech-workers-rs
git checkout $BRANCH
git pull origin $BRANCH

# Build BoringSSL
if [ ! -f "zig-tls-tunnel/vendor/boringssl/build/libssl.a" ]; then
    echo "🔨 Building BoringSSL..."
    cd zig-tls-tunnel/vendor/boringssl
    mkdir -p build && cd build
    cmake -GNinja -DCMAKE_BUILD_TYPE=Release -DCMAKE_C_COMPILER=clang -DCMAKE_CXX_COMPILER=clang++ ..
    ninja -j$(nproc)
    cd /content/ech-workers-rs
fi

# Build Zig
echo "🔨 Building Zig TLS Tunnel..."
cd zig-tls-tunnel
zig build -Doptimize=ReleaseFast
cd ..

# Build Rust
echo "🔨 Building Rust client..."
cd ech-workers-rs
cargo build --release 2>&1 | tail -5

# Strip
if [ -f "target/release/ech-workers-rs" ]; then
    strip target/release/ech-workers-rs
    echo ""
    echo "========================================"
    echo "✅ Build Complete!"
    echo "========================================"
    ls -lh target/release/ech-workers-rs
else
    echo "❌ Build failed"
fi